In [2]:
import os
import Bio
import pandas as pd

In [3]:
wd = "."
input_id = "faba_pandda_hits"
in_dir = os.path.join(wd, "input", input_id)
out_dir = os.path.join(wd, "output", input_id)
results_dir = os.path.join(out_dir, "results")

lig_data = pd.read_csv(os.path.join(results_dir, "{}_lig_data.csv".format(input_id)))

In [4]:
lig_data.head(3)

,struc_name,label_comp_id,label_asym_id,auth_seq_id
0,FabA-x0192-pandda-model.pdb,LIG,F,1
1,FabA-x0126-pandda-model.pdb,LIG,F,1
2,FabA-x0233-pandda-model.pdb,LIG,F,1


In [5]:
def get_swissprot(): 
    """
    Retrieves sequences and their data from Swiss-Prot

    :param db: absolute path to a fasta file containing sequences, Swiss-Prot database by default
    :type db: str
    :returns: dictionary containing the sequence id, description and sequence for all proteins in Swiss-Prot
    :rtpe: dict
    """
    swissprot_dict = Bio.SeqIO.parse(swissprot, "fasta")
    proteins = {}
    for protein in swissprot_dict:
        acc = protein.id.split("|")[1]
        proteins[acc] = {}
        proteins[acc]["id"] = protein.id
        proteins[acc]["desc"] = protein.description
        proteins[acc]["seq"] = protein.seq
    return proteins

In [16]:
def retrieve_mapping_from_struc(struc, uniprot_id, struc_dir, sifts_dir, swissprot):
    input_struct = os.path.join(struc_dir, struc)
    pdb_structure = PDBXreader(inputfile = input_struct).atoms(format_type = "pdb") # ProIntVar reads the local file
    
    seq_record = str(swissprot[uniprot_id]["seq"])
    pps = pdb_structure[pdb_structure.group_PDB == "ATOM"][['label_comp_id', 'label_asym_id', 'label_seq_id_full']].drop_duplicates().groupby('label_asym_id')  # groupby chain
    pdb_chain_seqs = [(chain, SeqUtils.seq1(''.join(seq['label_comp_id'].values)), seq['label_seq_id_full'].values) for chain, seq in pps] # list of tuples like: [(chain_id, chain_seq, [chain resnums])]
    alignments = [pairwise2.align.globalxs(str(seq_record),chain_seq[1], -5, -1) for chain_seq in pdb_chain_seqs] # list of lists of tuples containing SwissProt seq - PDB chain seq pairwise alignment
    
    maps = []
    for pdb_chain_seq, alignment in zip(pdb_chain_seqs, alignments):
        PDB_UniProt_map = pd.DataFrame([(i, x) for i, x in enumerate(alignment[0][1], start=1)],  # create aligned PDB sequences to dataframe
                                       columns=['UniProt_ResNum', 'PDB_ResName'])
        PDB_UniProt_map = PDB_UniProt_map.assign(UniProt_ResName = list(alignment[0][0]))
        PDB_index = PDB_UniProt_map.query('PDB_ResName != "-"').index
        PDB_UniProt_map = PDB_UniProt_map.assign(PDB_ResNum = pd.Series(pdb_chain_seq[2], index=PDB_index)) # adds PDB_ResNum column
        PDB_UniProt_map = PDB_UniProt_map.assign(PDB_ChainID = pd.Series(pdb_chain_seq[0], index=PDB_index)) # adds PDB_ChainId column
        maps.append(PDB_UniProt_map)
    prointvar_mapping = pd.concat(maps)
    prointvar_mapping = prointvar_mapping[['UniProt_ResNum','UniProt_ResName','PDB_ResName','PDB_ResNum','PDB_ChainID']]
    prointvar_mapping = prointvar_mapping[~prointvar_mapping.PDB_ResNum.isnull()]
    #prointvar_mapping = prointvar_mapping.fillna("null")
    prointvar_mapping.PDB_ResNum = prointvar_mapping.PDB_ResNum.astype(int)
    prointvar_mapping_csv = os.path.join(sifts_dir, "prointvar_mapping_" + struc + ".csv")
    prointvar_mapping.to_csv(prointvar_mapping_csv, index = False)
    return prointvar_mapping

In [9]:
import Bio.SeqIO

In [10]:
swissprot = "/cluster/gjb_lab/2394007/data/all_species/swissprot_rev_Nov21.fasta"

SP = get_swissprot()

In [11]:
SP["O33877"]

{'id': 'sp|O33877|FABA_PSEAE',
 'desc': 'sp|O33877|FABA_PSEAE 3-hydroxydecanoyl-[acyl-carrier-protein] dehydratase OS=Pseudomonas aeruginosa (strain ATCC 15692 / DSM 22644 / CIP 104116 / JCM 14847 / LMG 12228 / 1C / PRS 101 / PAO1) OX=208964 GN=fabA PE=1 SV=1',
 'seq': Seq('MTKQHAFTREDLLRCSRGELFGPGNAQLPAPNMLMIDRIVHISDVGGKYGKGEL...DSF', SingleLetterAlphabet())}

In [21]:
from prointvar.pdbx import PDBXreader
from Bio import SeqUtils
from Bio import pairwise2

In [22]:
a = retrieve_mapping_from_struc(
    "FabA-x0569-pandda-model.pdb",
    "O33877",
    os.path.join(out_dir, "supp_pdbs"),
    os.path.join(out_dir, "sifts"),
    SP
)

2023-09-04 16:55:41,444 prointvar    INFO     Parsing PDB atoms from lines...
2023-09-04 16:55:42,026 prointvar    INFO     PDBx removed existing hydrogens...
2023-09-04 16:55:42,031 prointvar    INFO     PDBx reset atom numbers...


In [23]:
a

,UniProt_ResNum,UniProt_ResName,PDB_ResName,PDB_ResNum,PDB_ChainID
1,2,T,T,2,A
2,3,K,K,3,A
3,4,Q,Q,4,A
4,5,H,H,5,A
5,6,A,A,6,A
6,7,F,F,7,A
7,8,T,T,8,A
8,9,R,R,9,A
9,10,E,E,10,A
10,11,D,D,11,A
